In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


### 1. Загрузить данные в pandas


In [5]:
data = pd.read_csv('u.data', sep= '\t', names = ['user_id', 'item_id', 'rating', 'timestamp'])
data.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
data.to_csv('u.data2', index=False)

### 2. Посчитайте среднюю оценку, которую поставил каждый пользователь

In [7]:
data.groupby('user_id').mean()['rating'].reset_index()

,user_id,rating
0,1,3.610294
1,2,3.709677
2,3,2.796296
3,4,4.333333
4,5,2.874286
...,...,...
938,939,4.265306
939,940,3.457944
940,941,4.045455
941,942,4.265823


### 3. Отсортируйте фильмы по количеству оценок

In [8]:
data.groupby('item_id').count()['rating'].sort_values(ascending=False).reset_index()

,item_id,rating
0,50,583
1,258,509
2,100,508
3,181,507
4,294,485
...,...,...
1677,1452,1
1678,1593,1
1679,1447,1
1680,814,1


### 4.Средствами pandasa соедините данные по оценкам с данными по пользователям (u.user)

In [9]:
users = pd.read_csv('u.user', sep= '|', names =['user_id', 'age', 'gender', 'occupation', 'zip_code'])
users.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [10]:
df = data.merge(users, on='user_id')

In [11]:
df.sample(5)

,user_id,item_id,rating,timestamp,age,gender,occupation,zip_code
38518,349,619,4,879466000,68,M,retired,61455
92266,872,932,4,888479498,19,F,student,74078
42422,389,483,5,879991535,44,F,writer,83702
66693,606,206,4,880927552,28,M,programmer,63044
85235,798,1239,4,875915965,40,F,writer,64131


### 5. Построить модель, которая предсказывает оценку для фильма и пользователя (воспользоваться RandomForestRegressor, тк мы решаем задачу регрессии)

In [12]:
from sklearn.ensemble import RandomForestRegressor

In [13]:
df_filtered = df[['user_id', 'item_id', 'rating']]

In [14]:
X = df_filtered.drop('rating', axis=1)
y = df_filtered['rating']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
rfr = RandomForestRegressor()

In [17]:
rfr.fit(X_train, y_train)

C:\Program Files (x86)\Soft data analyst\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [18]:
y_pred = rfr.predict(X_test)

### 6. Оценить качество регрессора на отложенной (train_test_split) выборке по одной из метрик на выбор - RMSE, RMAE, R^2

In [19]:
mean_absolute_error(y_test, y_pred)

0.907315

In [20]:
mean_squared_error(y_test, y_pred)

1.3171385000000002

In [21]:
r2_score(y_test, y_pred)

-0.045406301422758144

### Далее Spark 

In [65]:
items = pd.read_csv('u.item', encoding='cp1251', sep='|', names = ['item_id', 'movie title', 'release date', 'video release date',
              'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation',
              "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
              'Thriller', 'War', 'Western'])
items.head()

items.set_index('item_id',inplace=True)

items.shape

items_genres = items[items==1].stack().reset_index().drop(0,1)

items_genres.head()

items2 = items_genres.join(items, on = 'item_id', how='left')

items2 = items2[['item_id', 'level_1', 'movie title', 'release date']]

items2.columns = ['item_id', 'genre', 'movie_title', 'release_date']

items2.to_csv('u.item2', index=False)

https://colab.research.google.com/drive/1TUI6ExrcU2gkEayWpKX0ki0rXPUWKvqc